# Englsih Premier League 2018/2019 seasons prediction

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

%matplotlib inline

In [2]:
df = pd.read_csv(r'D:\git_space\data\Premier-League_18-19seasons_matches.csv')
df.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1533927600,Aug 10 2018 - 7:00pm,complete,74439,Manchester United,Leicester City,Andre Marriner,1,0.0,0.0,...,1.37,4.98,9.81,1.33,2.00,3.60,7.50,2.05,1.69,Old Trafford (Manchester)
1,1533987000,Aug 11 2018 - 11:30am,complete,51749,Newcastle United,Tottenham Hotspur,Martin Atkinson,1,0.0,0.0,...,4.51,3.77,1.84,1.31,1.95,3.50,5.75,1.83,1.87,St. James' Park (Newcastle upon Tyne)
2,1533996000,Aug 11 2018 - 2:00pm,complete,10353,AFC Bournemouth,Cardiff City,Kevin Friend,1,0.0,0.0,...,2.03,3.51,3.96,1.31,1.95,3.45,6.70,1.83,1.83,Vitality Stadium (Bournemouth- Dorset)
3,1533996000,Aug 11 2018 - 2:00pm,complete,24821,Fulham,Crystal Palace,Mike Dean,1,0.0,0.0,...,2.31,3.44,3.26,1.28,1.87,3.20,7.00,1.71,2.00,Craven Cottage (London)
4,1533996000,Aug 11 2018 - 2:00pm,complete,24121,Huddersfield Town,Chelsea,Chris Kavanagh,1,0.0,0.0,...,7.47,4.27,1.51,1.29,1.91,3.30,6.95,2.00,1.71,John Smith's Stadium (Huddersfield- West Yorks...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 64 columns):
timestamp                              380 non-null int64
date_GMT                               380 non-null object
status                                 380 non-null object
attendance                             380 non-null int64
home_team_name                         380 non-null object
away_team_name                         380 non-null object
referee                                380 non-null object
Game Week                              380 non-null int64
Pre-Match PPG (Home)                   380 non-null float64
Pre-Match PPG (Away)                   380 non-null float64
home_ppg                               380 non-null float64
away_ppg                               380 non-null float64
home_team_goal_count                   380 non-null int64
away_team_goal_count                   380 non-null int64
total_goal_count                       380 non-null int64
total_goals_

In [5]:
df.isnull().sum()

timestamp         0
date_GMT          0
status            0
attendance        0
home_team_name    0
                 ..
odds_ft_over35    0
odds_ft_over45    0
odds_btts_yes     0
odds_btts_no      0
stadium_name      0
Length: 64, dtype: int64